<a href="https://colab.research.google.com/github/nattrio/NF-kB-prediction/blob/main/get_smiles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
downloaded = drive.CreateFile({'id':"1qhUkr8R7omHFN_2KRQlIbrAAtkgNN5C2"})   # replace the id with id of file you want to access
downloaded.GetContentFile('GeneID_4790_bioactivity_gene.csv')        # replace the file name with your file

In [ ]:
import pandas as pd
df = pd.read_csv("GeneID_4790_bioactivity_gene.csv", header=0)
df.info()
# df.groupby(['activity']).count()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (14,21) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 238850 entries, 0 to 238849
Data columns (total 24 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   baid        238850 non-null  int64  
 1   activity    238850 non-null  object 
 2   aid         238850 non-null  int64  
 3   sid         238850 non-null  int64  
 4   mid         238850 non-null  int64  
 5   cid         238850 non-null  int64  
 6   geneid      238850 non-null  int64  
 7   taxid       238850 non-null  int64  
 8   pmid        1036 non-null    float64
 9   aidtype     238850 non-null  object 
 10  aidmdate    238850 non-null  int64  
 11  hasdrc      238850 non-null  int64  
 12  rnai        238850 non-null  int64  
 13  protacxn    238850 non-null  object 
 14  acname      25579 non-null   object 
 15  acvalue     4679 non-null    float64
 16  aidsrcname  238850 non-null  object 
 17  aidname     238850 non-null  object 
 18  cmpdname    238850 non-null  object 
 19  ta

In [ ]:
df = df.loc[df['activity'].isin(["Active", "Inactive"])]
# df.info()

In [ ]:
def isActive(str):
    if str == "Active":
        return 1
    elif str == "Inactive": 
        return 0

In [ ]:
df["class"] = df["activity"]
class_data = df['class'].to_numpy()

for i in range(len(class_data)):
    class_data[i] = isActive(class_data[i])

print(class_data)

[1 1 1 ... 0 0 0]


In [ ]:
df = df[["cid", "class"]]

In [ ]:
!pip install pubchempy
import os
import time
import pandas as pd
import pubchempy as pcp
print(os.getcwd())

  Created wheel for pubchempy: filename=PubChemPy-1.0.4-cp36-none-any.whl size=13825 sha256=978720fee1a669d4a2d2532a00074595cb9af94c6589793e4306576031f090b9
  Stored in directory: /root/.cache/pip/wheels/10/4d/51/6b843681a9a5aef35f0d0fbce243de46f85080036e16118752
Successfully built pubchempy
/content


In [ ]:
# df_active = df.loc[df['class'] == 1].tail(312)
# df_active

In [ ]:
# 228790
df_inactive = df.loc[df['class'] == 0].tail(98790)
df_inactive = df_inactive.head(30000)
df_inactive

,cid,class
139458,2079240,0
139459,2079294,0
139460,2079334,0
139461,2079393,0
139462,2079436,0
...,...,...
169453,11958699,0
169454,11958701,0
169455,11958702,0
169456,11958708,0


In [ ]:
def to_smiles(data):
    smiles = []
    i = 0
    for c in data.cid:
        out = pcp.Compound.from_cid(c)
        smiles.append(out.isomeric_smiles)
        #print out.cid
        #print out.isomeric_smiles
        i = i+1
        if i % 100 == 0:
          print(i)
        elif i % 10 == 0:
          print("*", end = '')
        else:
          print(".", end = '')
        # time.sleep(0.5)
    return smiles

In [ ]:
# df_active['smiles'] = to_smiles(df_active)
# df_active

In [ ]:
df_inactive['smiles'] = to_smiles(df_inactive)
df_inactive

.........*.........*.........*.........*.........*.........*.........*.........*.........*.........100
.........*.........*.........*.........*.........*.........*.........*.........*.........*.........200
.........*.........*.........*.........*.........*.........*.........*.........*.........*.........300
.........*.........*.........*.........*.........*.........*.........*.........*.........*.........400
.........*.........*.........*.........*.........*.........*.........*.........*.........*.........500
.........*.........*.........*.........*.........*.........*.........*.........*.........*.........600
.........*.........*.........*.........*.........*.........*.........*.........*.........*.........700
.........*.........*.........*.........*.........*.........*.........*.........*.........*.........800
.........*.........*.........*.........*.........*.........*.........*.........*.........*.........900
.........*.........*.........*.........*.........*.........*.........*...

In [ ]:
from google.colab import files
# df_active.to_csv('df_active.csv', index=False)
df_inactive.to_csv('df_inactive.csv', index=False)
# files.download('df_active.csv')
files.download('df_inactive.csv')